# IndicTrans2 HF Inference

We provide an example notebook on how to use our IndicTrans2 models which were originally trained with the fairseq to HuggingFace transformers for inference purpose.


## Setup

Please run the cells below to install the necessary dependencies.


In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

**IMPORTANT : Restart your run-time first and then run the cells below.**

## Inference


In [1]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [2]:
!pip install sacrebleu

In [3]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

### English to Indic Example


In [4]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)

en_sents = [
    # trying out different locations but always gives जाता था in male tone.
    # "When I was young, I used to go to the park every day.",
    # "When I was young, I used to go to the mall every day.",
    # "When I was young, I used to go to the school every day.",
    # "When I was young, I used to go to the garden every day.",
    # "When I was young, I used to go to the library every day.",
    # "When I was young, I used to go to the parlour every day.",
    # "When I was young, I used to go to the salon every day.",
    # "When I was young, I used to go to the hospital every day.",
    # # trying spl symbols. its ignoring them. DOESN'T convert Numbers to Hindi
    # "*** let's meet at 1:30pm today.",
    # "*** let's meet at 1:30pm today. ***",
    # #famous tag line "Paytm Karo" could not detect?? - detected but politely
    # "You owe me ₹1234. Paytm me.",
    # "You owe me ₹1234. Do Paytm me.",
    # "Good morning",

    # #sound rude but LLM can't
    # "Return my money! Else you are doomed!!!",
# we need to convert "Indian" english and not actual American native english to Inidc lang



#can do joint sentences with diff context
#     "He has many old books, which he inherited from his ancestors. I can't figure out how to solve my problem.",
#     "She is very hardworking and intelligent, which is why she got all the good marks.",
#     "She is very hardworking and inteLLigent, which is y she got all the g00d marks.",

#     "She is very hardworking and inteLLigent, which is Y she got all the g00d marks.",
#     "We watched a new movie last week, which was very inspiring.",
#     "If you had met me at that time, we would have gone out to eat.",
#     "She went to the market with her sister to buy a new sari.",
#     "Raj told me that he is going to his grandmother's house next month.",
#     "All the kids were having fun at the party and were eating lots of sweets.",
#     "My friend has invited me to his birthday party, and I will give him a gift.",
# # for healthcare applications
#     "My back is hurting real bad",
#     "The pus oozes out from the swollen thing that I have on my left thumb",
#     "I can't breathe properly when I walk fast",
#     "I am not feeling good, I just don't feel like going out of my room",
#     "I think I am pregnant. Can you please check?",
#     "I think he is pregnant",
#     "I am suffering from a severe hairloss. Dr. please suggest a solution for me",
#     "I want my xray to be done for my leg",
#     "My pinky finger hurts badly", #
#     "Ouch my calf hurts!!!!!", #
#     "My cow gave birth to a little calf yesterday and we named him moooo",
#     "My heels really hurts",
#     "The wound on my ankle is still fresh",
#     "Intestine, Pancreas, bladder, spleen, hips, wrist , elbow, palm, upper arm, forearm, shoulder",
#     "Adam's Apple" ,#
#     "Abdomen , chest, spine, stomach", # both abdomen and stomach = पेट
#     "Glands such as the pituitary, thyroid, and adrenal glands",
#     #Identified sole is being used as a noun or adjective
#     "My sole is aching real bad",
    # "The sole reason I left my job is to go for higher studies" ,
    # "I missed my flight",
    # "Miss Sofia won the contest",
    # "I miss you Sofia",
    # "I feel truly sorry for you",

    #####PROVERB##########
    # "It's raining cats and dogs",#
    # "His wardrobe was at sixes and sevens",#
    # "She was over the moon when she got the job offer.",
    # "Don't count your chickens before they hatch.",
    # "He's feeling under the weather today.",
    # "She's burning the midnight oil to finish her project on time.",
    # "I'm feeling on top of the world after acing my exam.",
    # "He's in hot water with his boss after missing the deadline.",
    # "They're like two peas in a pod, always together.",
    # "I'm on cloud nine after hearing the good news.",
    # "She's walking on eggshells around her temperamental boss.",
    # "He's trying to kill two birds with one stone by multitasking.",
    # "She's been on pins and needles waiting for the results of her interview."
    # "He's the black sheep of the family, always causing trouble.",
    # "They hit the nail on the head with their analysis of the situation.",
    # "She's feeling like a fish out of water in the new city.",
    # "He's barking up the wrong tree if he thinks he can convince her.",
    # "I'm feeling on cloud nine after winning the competition.",
    # "She's burning the candle at both ends trying to balance work and school.",
    # "He's as cool as a cucumber even under pressure.",
    # "I'm in the doghouse with my parents after coming home late.",
    # "She's got a chip on her shoulder about not being promoted.",
    # "He's as stubborn as a mule when it comes to changing his mind",
    # "She's as blind as a bat without her glasses",
    # "He's got a bee in his bonnet about recycling",
    # "She's as busy as a bee preparing for the upcoming event",
    # "He's as fit as a fiddle for someone his age",
    # "She's like a bull in a china shop when she's in a hurry",
    # "He's as sly as a fox when it comes to negotiations",
    # "She's as quiet as a mouse during the meeting",
    # "He's like a fish out of water in social gatherings",
    # "She's as sharp as a tack when it comes to solving puzzles",
    # "He's as busy as a bee during tax season",
    # "She's as quiet as a mouse during the movie",
    # "He's as brave as a lion when facing challenges",
    # "She's as fast as lightning on the track",
    # "He's as cold as ice when angry",
    # "She's as wise as an owl when giving advice",
    # "He's as happy as a clam on vacation",
    # "She's as gentle as a lamb with children",
    # "He's as strong as an ox when lifting weights",
    # "She's as graceful as a swan on the dance floor",
    # "He's as slippery as an eel in negotiations",
    # "She's as light as a feather on her feet",
    # "He's as sharp as a razor when it comes to wit",
    # "She's as bright as a button in class",
    # "He's as busy as a beaver building his business",
    # "She's as cool as a cucumber under pressure",
    # "He's as stubborn as a mule about his opinions",
    # "She's as busy as a bee in the garden",
    # "He's as happy as a lark on his birthday",
    # "She's as blind as a bat without her glasses",
    # "He's as quiet as a mouse during the lecture",
    # "She's as brave as a lion when standing up for herself",
    # "He's as slow as molasses in the morning",
    # "She's as busy as a squirrel gathering nuts",
    # "He's as slippery as a fish in the courtroom",
    # "She's as happy as a clam at the beach",
    # "He's as fit as a fiddle after his workout",
    # "She's as sweet as pie when she wants something",
    # "He's as strong as an ox when lifting weights",
    # "She's as fast as lightning on the soccer field",
    # "He's as stubborn as a bull when it comes to change",
    # "She's as quiet as a church mouse during the ceremony",
    # "He's as hungry as a bear after his long hike",
    # "She's as busy as a hen with her chicks",
    # "He's as happy as a pig in mud on his day off",
    # "She's as sharp as a tack in her observations",
    # "He's as quiet as a tomb during the test",
    # "She's as sly as a fox when it comes to tricks",
    # "He's as fierce as a tiger when defending his territory",
    # "She's as wise as an owl in her decisions",
    # "He's as light as a feather when he's happy",
    # "She's as red as a lobster after a day in the sun",
    # "He's as strong as a bull in the gym",
    # "She's as busy as a jackrabbit hopping from task to task",
    # "He's as happy as a clam at high tide",
    # "She's as slow as a turtle in making decisions",
    # "He's as happy as a lark in the morning",
    # "She's as proud as a peacock of her accomplishments",
    # "He's as sly as a snake in his dealings",
    # "She's as graceful as a swan on the dance floor",
    # "He's as busy as a hornet fixing his car",
    # "She's as quiet as a lamb in the library",
    # "He's as tough as nails when facing adversity",
    # "She's as fast as a cheetah on the track",
    # "He's as strong as an ox in the weight room",
    # "She's as happy as a pig in slop with her new toy",
    # "He's as busy as a dog chasing its tail",
    # "She's as brave as a lioness protecting her cubs",
    # "He's as stubborn as a bulldog with his opinions",
    # "She's as quiet as a mouse in the corner",
    # "She's walking on eggshells around her boss after the mistake",
    # "He's trying to kill two birds with one stone by finishing his work early",
    # "She's feeling under the weather, so she's staying home from work",
    # "He's in hot water with his parents for breaking the rules",
    # "She's on cloud nine after receiving the promotion",
    # "He's feeling like a fish out of water in the new social group",
    # "She's burning the candle at both ends to meet the deadline",
    # "He's as quiet as a mouse during the meeting",
    # "She's like a bull in a china shop when she's stressed",
    # "He's feeling like the black sheep of the family after the argument",
    # "She's as busy as a bee organizing the event",
    # "He's as happy as a clam now that he's retired",
    # "She's as fit as a fiddle after her morning jog",
    # "He's as stubborn as a mule about his opinions",
    # "She's as sharp as a tack when it comes to solving problems",
    # "He's as cool as a cucumber under pressure",
    # "She's like a fish out of water in the big city",
    # "He's as busy as a beaver building his business",
    # "She's as happy as a lark on her wedding day",
    # "He's as busy as a hornet with his chores",
    # "She's as busy as a beaver renovating her house",
    # "He's as quiet as a mouse in the library",
    # "She's as fit as a fiddle after her morning yoga session",
    # "He's as happy as a clam at the family reunion",
    # "She's as stubborn as a mule when it comes to learning new things",
    # "He's as cool as a cucumber during the job interview",
    # "She's like a fish out of water at the fancy gala",
    # "He's as sharp as a tack in his presentation",
    # "She's as busy as a bee planning her wedding",
    # "He's as happy as a lark on his vacation",
    # "She's as blind as a bat without her morning coffee",
    # "He's as fit as a fiddle after his workout at the gym",
    # "She's as busy as a beaver preparing for the exam",
    # "He's as quiet as a mouse when he's focused on reading",
    # "She's as happy as a clam with her new puppy",
    # "He's as stubborn as a mule about sticking to his routine",
    # "She's as cool as a cucumber despite the stressful situation",
    # "He's as busy as a beaver organizing his workspace",
    # "She's as sharp as a tack in her problem-solving skills",
    # "He's as quiet as a mouse when he's lost in thought",
    # "The melody danced in her head, a catchy tune stuck on repeat."
    # "He spoke with a silver tongue, his words smooth and persuasive.",
    # "The city lights twinkled like a million fireflies against the inky sky.",
    # "She felt a surge of adrenaline, her heart pounding like a drum.",
    # "The old book held a world of secrets within its worn pages.",
    # "The waves crashed against the shore, a rhythmic roar echoing in the air.",
    # "He felt a million miles away, lost in his thoughts.",
    # "The sunset painted the sky in a kaleidoscope of colors, a breathtaking spectacle.",
    # "The old woman's smile was like sunshine, radiating warmth and kindness.",
    # "He had a sixth sense about danger, a feeling of unease gnawing at him.",
    # "Laughter filled the room, a contagious symphony of joy.",
    # "The ticking clock seemed to mock him, each second dragging by like an eternity.",
    # "The aroma of freshly cut grass hung in the air, a reminder of summer days.",
    # "She felt a wave of nausea wash over her, a churning in her stomach.",
    # "The weight of the world seemed to be on his shoulders, a heavy burden to bear.",
    # "Time seemed to stand still, the moment suspended in a bubble of anticipation.",
    # "The winding path beckoned her forward, a promise of adventure.",
    # "His eyes sparkled with mischief, a hint of trouble brewing.",
    # "The silence was broken only by the gentle patter of rain on the roof.",
    # "She danced with abandon, her body a canvas of movement and freedom.",
    # "The library was a haven of knowledge, its shelves overflowing with stories waiting to be discovered.",
    # "The old man reminisced about his past, his voice a whisper filled with nostalgia.",
    # "The aroma of coffee filled the air, a comforting scent that chased away the morning chill.",
    # "He wore a cloak of secrecy, his true motives hidden from view.",
    # "The stars winked down at them, like a million tiny diamonds scattered across the velvet night.",
    # "The taste of victory was sweet, a reward for all his hard work.",
    # "She spoke with a fiery passion, her words igniting a spark in the hearts of her listeners.",
    # "The old building loomed over them, its dark windows like vacant eyes.",
    # "The first rays of dawn peeked over the horizon, painting the sky with a soft pink glow.",
    # "He felt a sense of calm wash over him, the tension of the day finally fading away.",
    # "The melody was haunting, a melancholic tune that tugged at your heartstrings.",
    # "He spoke with a forked tongue, his words twisting the truth to suit his agenda.",
    # "The city lights twinkled like a fallen galaxy, a breathtaking display of human ingenuity.",
    # "She felt a pang of guilt, a gnawing feeling that wouldn't let go.",
    # "The old book reeked of must and decay, a testament to its passage through time.",
    # "The waves crashed against the shore with a thunderous roar, a powerful display of nature's raw force.",
    # "He felt a deep sense of connection with nature, a feeling of belonging.",
    # "The sunrise painted the sky with streaks of gold and orange, a magnificent display of color.",
    # "The young girl's laughter was like music to his ears, a sound filled with innocence and joy.",
    # "His confidence was infectious, his belief in himself radiating outward.",
    # "The air crackled with anticipation, the tension thick enough to cut with a knife.",
    # "The ticking clock was a constant reminder of the time slipping away.",
    # "The aroma of freshly baked cookies filled the air, a warm and inviting aroma that tickled the senses.",
    # "He felt a sense of dread, a feeling that something bad was about to happen.",
    # "The weight of the world seemed to be on her shoulders, a heavy burden that threatened to crush her.",
    # "Time seemed to stretch on forever, each minute dragging by like an eternity.",
    # "The winding path through the forest was shrouded in mystery, a dark and unknown world waiting to be explored.",
    # "Her eyes welled up with tears on the verge of spilling over.",
    # "The silence was broken",



    # ################GEMINI GENERATED###################
    # "She laughed so hard her sides were splitting.",
    # "Time flies when you're having fun but crawls when you're bored.",
    # "He was feeling under the weather so he called it a day.",
    # "The traffic was bumper to bumper, a sea of brake lights.",
    # "Her mind was a whirlwind of thoughts making it hard to focus.",
    # "He felt like a fish out of water lost and uncomfortable.",
    # "The news spread like wildfire reaching everyone in no time.",
    # "The old house creaked and groaned like a haunted mansion.",
    # "My wife won't let me speak to my mother",#
    # "My husband won't let me speak to my mother",#
    # "My husband took me to Musoorie for holidays",
    # "My wife took me to Thailand for holidays",
    # "My wife gifted me a INR 100 thousand purse",
    # "She gifted me a INR 100 thousand purse",
    # "He gifted me a INR 100 thousand purse",
    # "I have got to buy a computer with larger RAM",
    # "Ram of my new computer is slightly bigger than my old computer",
    # "My Algorithm is better than yours",
    # "Algorithm: A step-by-step set of instructions or rules followed to perform a task or solve a problem.",
    # "Big Data: Extremely large and complex data sets that cannot be easily managed or processed with traditional data processing tools.",
    # "Machine Learning: A subset of artificial intelligence that focuses on the development of algorithms and statistical models that enable computers to improve their performance on a task through experience.",
    # "Data Mining: The process of discovering patterns and knowledge from large amounts of data.",
    # "Data Science: The interdisciplinary field that uses scientific methods, processes, algorithms, and systems to extract insights and knowledge from structured and unstructured data.",
    # "Data Analytics: The process of examining, cleaning, transforming, and modeling data to extract useful information, draw conclusions, and support decision-making.",
    # "Artificial Intelligence (AI): The simulation of human intelligence in machines that are programmed to think and learn like humans.",
    # "High-Performance Computing (HPC): The use of supercomputers and parallel processing techniques to solve complex computational problems.",
    # "Cloud Computing: The delivery of computing services, including storage, processing power, and analytics, over the internet.",
    # "Distributed Computing: A field of computer science that deals with designing and implementing systems that involve multiple interconnected computers working together.",
    # "Parallel Computing: The simultaneous execution of multiple calculations or processes to solve a complex problem more efficiently.",
    # "Deep Learning: A subset of machine learning that involves neural networks with multiple layers (deep neural networks) to learn and make decisions.",
    # "Natural Language Processing (NLP): The field of artificial intelligence that focuses on the interaction between computers and humans using natural language.",
    # "Predictive Modeling: The process of creating and testing a model to predict future outcomes based on historical data.",
    # "Quantum Computing: A type of computing that uses the principles of quantum mechanics to perform calculations, offering potential advantages for certain types of problems.",
    # "Computational Biology: The application of computational techniques to the understanding and modeling of biological systems.",
    # "Network Analysis: The study of relationships and connections between nodes in a network, often represented as graphs.",
    # "Data Visualization: The representation of data in graphical or visual formats to help people understand the patterns, trends, and insights within the data.",
    # "Hadoop: An open-source framework for distributed storage and processing of large data sets.",
    # "Cybersecurity: The practice of protecting computer systems, networks, and data from security breaches and unauthorized access.",
    # "My brother-in-law helped me in cleaning the space for the guests",
    # "My sister-in-law helped me in cleaning the space for the guests",
    # "My grandmother cooks delicious meals to us",
    # "My grandfather is a man of intelligence",
    # "We went to our grandmother's house yesterday",
    # "We will go to our aunt's today",
    # "My niece is so cute that I want to squish her face everytime I see her",# neice = nees(in hindi)
    # "My nephew started going to school last week",
    # "My aunt goes to gym everyday",
    # "My aunt cooks really delicious meals", #aunt -- chachi, uncle -- chacha
    # "My sister-in-law told me that she is thinking of joining the gym", #sister-in-law -- saali, #brother-in-law-- jeeja
    # "My uncle asked me to stay",
    # "My maternal gradmother cooks delicious food for us",
    # "My maternal uncle owns a shoe store",
    # "My brother loves eating chocolate",
    # "My brother's wife loves chocolate",
    # "My cousins danced at my brother's wedding",
    # "She sells seashells on the sea shore",
    # "My sister in law is equivalent to my mother",#
    # "My mother is my grandmother's daughter ",#
    # "My washbasin is white in color",
    # "My brother's mother's daughter's only brother's wife's son's mother is my sister-in-law",#
    # "my brother's wife is my sister in law",#
    # "I have always wanted to go to Saharanpur during winters",
    # "Have you ever had Chhole bhature with Lassi in New Delhi?",
    # "I live in Jafar Nawaz",
    # "My father owns a shop named Naveen Stationers in Saharanpur",
    # "Hey, did you watch the movie Kesariya?",
    # "I can't believe you have never watched Kabhi khushi kabhi gum.",
    # "Did you take a bath today? You are smelling awful today.",
    # "You are a very reliable person in my life who I can tell all my secrets to."
    # "You know , the panda who keeps an eye on my drinking habits",
    # "But I will also be keeping an eye on you",
    # "I will keep an eye on him , ’ reassured Jack",
    # "Moreover , international operations director Richard Ferry claims that the Bracknell , Berkshire-based company 's services are unique — although most rivals can offer centralised expertise , he says , no-one else can monitor computer systems remotely , or keep an eye on such critical issues as temperature and air conditioning levels at any given customer site",
    # "Once Duncan had managed to fire off a report and stop his watch-replacement from stumbling on the terrorists , he had settled down to keep an eye on the gang",
    # "So we must keep an eye on Bruno Casanova .",
    # "Keep your eyes peeled for bandits , there might be a few up tonight",
    # "As well as keeping their eyes peeled for surface coins and jewellery , beachcombers also look for non-metal items on their foreshore hunting grounds",
    # " My overseers keep their eyes peeled here , I can tell you .",
    # "We hope you find the current issue still of interest and we plan to keep up the good work during the coming year seeking out happenings/events/ stories to retell for you , but as it is your newspaper do contact us with your own ideas and suggestions at any time .",
    # "P.S. Tennis World is brilliant — keep up the good work .",
    # "I will try and keep up the good work .",
    # "Please keep up the good work by introducing all your new class members to Medau membership ; forms with the new prices for 1992 are available from the office .",
    # "Many thanks , and keep up the good work !",
    # "However , Wanderers are true amateurs on the field and will no doubt battle until they give up , so let's hope Athletico can keep up the good work and send them home with three lost points and the Samaritans ’ telephone number .",
    # "Thanks to all those who have informed me of sightings of wildlife ; please keep up the good work and contact me in the Library & information Service",
    # "What a lack of self-confidence this implies : the present wants both to patronise the past by adjudicating on its political acceptability , and also to be flattered by it , to be patted on the back and told to keep up the good work",
    # "Please keep up the good work , your mag is great",
    # "Please keep up the good work",
    # "When we 're ready , that is at zero hour , he will climb out of the night lavatory window and first make sure that the immediate area round the block is clear of goons .",
    # "It was zero hour for the well armed and equipped two-man penetration team .",
    # "Grant watched them go , chatting amiably and swapping friendly banter as they went , easing their inner tensions as they moved nearer to zero hour",
    # "We want everybody to be ready to fall into their places five minutes before zero hour . ",
    # "Similarly any prerecorded voices , sound effects and incidental music needed to be prepared and in the hands of the Grams Operator , in order of cueing , before zero hour on the big day",
    # "As zero hour was reached , and punctually to the very minute our barrage lifted , through my glasses I could clearly observe our skirmishers leave their trenches and move steadily forward ; here and there I could even distinguish the smoke puffs of bursting bombs",
    # "The hovercraft has always suffered from the fact that it is neither fish nor fowl .",
    # "A bird in hand is worth two in bush"

    "A bad penny always turns up",
    "A barking dog never bites",
    "A bird in the hand is worth two in the bush",
    "A cat may look at a king",
    "A chain is only as strong as its weakest link",
    "A change is as good as a rest",
    "A dog is a man's best friend",
    "A drowning man will clutch at a straw",
    "A fish always rots from the head down",
    "A fool and his money are soon parted",
    "A friend in need is a friend indeed",
    "A golden key can open any door",
    "A good beginning makes a good ending",
    "A good man is hard to find",
    "A house divided against itself cannot stand",
    "A house is not a home",
    "A journey of a thousand miles begins with a single step",
    "A leopard cannot change its spots",
    "A little knowledge is a dangerous thing",
    "A little learning is a dangerous thing",
    "A little of what you fancy does you good",
    "A man is known by his friends",
    "A man who is his own lawyer has a fool for his client",
    "A miss is as good as a mile",
    "A new broom sweeps clean",
    "A nod's as good as a wink to a blind horse",
    "A penny saved is a penny earned",
    "A person is known by the company he keeps",
    "A picture paints a thousand words",
    "A place for everything and everything in its place",
    "A poor workman always blames his tools",
    "A problem shared is a problem halved",
    "A prophet is not recognized in his own land",
    "A rising tide lifts all boats",
    "A rolling stone gathers no moss",
    "A soft answer turneth away wrath",
    "A stitch in time saves nine",
    "A swarm in May is worth a load of hay; a swarm in June is worth a silver spoon; but a swarm in July is not worth a fly",
    "A thing of beauty is a joy forever",
    "A time and a place for everythingre's",
    "A trouble shared is a trouble halved",
    "A volunteer is worth twenty pressed men",
    "A watched pot never boils",
    "A woman is only a woman, but a good cigar is a smoke",
    "A woman's place is in the home",
    "A woman's work is never done",
    "A word to the wise is enough",
    "Absence makes the heart grow fonder",
    "Absolute power corrupts absolutely",
    "Accidents will happen (in the best-regulated families).",
    "Actions speak louder than words",
    "Adversity makes strange bedfellows",
    "After a storm comes a calm",
    "Age of miracles is past",
    "Ain't over till the fat lady sings - It",
    "All good things come to he who waits",
    "All good things must come to an end",
    "All is grist that comes to the mill",
    "All publicity is good publicity",
    "All roads lead to Rome",
    "All that glisters is not gold",
    "All that glitters is not gold",
    "All the world loves a lover",
    "All things come to those that wait",
    "All things must pass",
    "All work and no play makes Jack a dull boy",
    "All you need is love",
    "All's fair in love and war",
    "All's for the best in the best of all possible worlds",
    "All's well that ends well",
    "A miss is as good as a mile",
    "An apple a day keeps the doctor away",
    "An army marches on its stomach",
    "An Englishman's home is his castle",
    "An exception to every rulere's",
    "Always more fish in the seare's",
    "An eye for an eye, a tooth for a tooth",
    "An ill wind that blows no one any good - It's",
    "An ounce of prevention is worth a pound of cure",
    "Another day, another dollar",
    "Any port in a storm",
    "Appearances can be deceptive",
    "Apple a day keeps the doctor awayn",
    "Apple never falls far from the tree",
    "April is the cruellest month",
    "April showers bring forth May flowers",
    "Army marches on its stomachn",
    "As thick as thieves",
    "As you make your bed, so you must lie upon it",
    "As you sow so shall you reap",
    "Ashes to ashes dust to dust",
    "Ask a silly question and you'll get a silly answer",
    "Ask no questions and hear no lies",
    "Attack is the best form of defence",
    "Bad money drives out good",
    "Bad news travels fast",
    "Bad penny always turns up",
    "Barking dog never bites",
    "Be careful what you wish for",
    "Beat swords into ploughshares",
    "Beauty is in the eye of the beholder",
    "Beauty is only skin deep",
    "Beggars should not be choosers",
    "Behind every great man there's a great woman",
    "Best defence is a good offence",
    "Best is the enemy of the good",
    "Best-laid schemes of mice and men gang aft agley",
    "Best things in life are free",
    "Better late than never",
    "Better safe than sorry",
    "Better the Devil you know than the Devil you don't",
    "Best to be on the safe side - It's",
    "Better to give than to receive - It's",
    "Better to have loved and lost than never to have loved at all - It's",
    "Better to light a candle than to curse the darkness - It's",
    "Better to remain silent and be thought a fool that to speak and remove all doubt",
    "Better to travel hopefully than to arrive - It's",
    "Between two stools one falls to the ground",
    "Beware of Greeks bearing gifts",
    "Beware the Ides of March",
    "Big fish eat little fish",
    "Big fleas have little fleas upon their back to bite 'em",
    "Bigger, the better",
    "Bigger they are, the harder they fall",
    "Bird in the hand is worth two in the bush",
    "Birds of a feather flock together",
    "Blessed are the peacemakers",
    "Blood is thicker than water",
    "Brevity is the soul of wit",
    "Build a better mousetrap and the world will beat a path to your door",
    "Business before pleasure",
    "Caesar's wife must be above suspicion",
    "Chain is only as strong as its weakest link",
    "Change is as good as a rest",
    "Charity begins at home",
    "Charity covers a multitude of sins",
    "Cheaters never win and winners never cheat",
    "Cheats never prosper",
    "Child is father to the man",
    "Children and fools tell the truth",
    "Children should be seen and not heard",
    "Christmas comes but once a year",
    "Cleanliness is next to godliness",
    "Clothes maketh the man",
    "Cobbler always wears the worst shoes",
    "Cold hands, warm heart",
    "Comparisons are odious",
    "Count your blessings",
    "Course of true love never did run smooth",
    "Cowards may die many times before their death",
    "Crime doesn't pay",
    "Curiosity killed the cat",
    "Customer is always right",
    "Cut your coat to suit your cloth",
    "Darkest hour is just before the dawn",
    "Dead men tell no tales",
    "Devil is in the details",
    "Devil looks after his own",
    "Devil makes work for idle hands to do",
    "Devil take the hindmost",
    "Different strokes for different folks",
    "Discretion is the better part of valour",
    "Distance lends enchantment to the view",
    "Do as I say, not as I do",
    "Do as you would be done by",
    "Do unto others as you would have them do to you",
    "Dog is a man's best friend",
    "Don't bite the hand that feeds you",
    "Don't burn your bridges behind you",
    "Don't cast your pearls before swine",
    "Don't change horses in midstream",
    "Don't count your chickens before they are hatched",
    "Don't cross the bridge till you come to it",
    "Don't cut off your nose to spite your face",
    "Don't get mad, get even",
    "Don't get your knickers in a twist",
    "Don't keep a dog and bark yourself",
    "Don't leave your manners on the doorstep",
    "Don't let the bastards grind you down",
    "Don't let the cat out of the bag",
    "Don't let the grass grow under your feet",
    "Don't look a gift horse in the mouth",
    "Don't meet troubles half-way",
    "Don't mix business with pleasure",
    "Don't put all your eggs in one basket",
    "Don't put the cart before the horse",
    "Don't put new wine into old bottles",
    "Don't rock the boat",
    "Don't shoot the messenger",
    "Don't spoil the ship for a ha'porth of tar",
    "Don't shut the stable door after the horse has bolted",
    "Don't sweat the small stuff",
    "Don't throw pearls to swine",
    "Don't teach your Grandma to suck eggs",
    "Don't throw good money after bad",
    "Don't throw the baby out with the bathwater",
    "Don't trust anyone over thirty",
    "Don't try to run before you can walk",
    "Don't try to walk before you can crawl",
    "Don't upset the apple-cart",
    "Don't wash your dirty linen in public",
    "Doubt is the beginning not the end of wisdom",
    "Drowning man will clutch at a straw",
    "Early to bed and early to rise, makes a man healthy, wealthy and wise",
    "Each to their own",
    "Early bird catches the worm",
    "East is east, and west is west",
    "East, west, home's best",
    "Easy come, easy go",
    "Easy to be wise after the event - It's",
    "Eat, drink and be merry, for tomorrow we die",
    "Empty vessels make the most noise",
    "End justifies the means",
    "English are a nation of shopkeepers",
    "Englishman's home is his castlen",
    "Enough is as good as a feast",
    "Enough is enough",
    "Even a worm will turn",
    "Every cloud has a silver lining",
    "Every dog has its day",
    "Every little helps",
    "Every man for himself, and the Devil take the hindmost",
    "Every man has his price",
    "Every picture tells a story",
    "Every stick has two ends",
    "Everyone wants to go to heaven but nobody wants to die",
    "Everything comes to him who waits",
    "Exception which proves the rule",
    "Eye for an eye, a tooth for a toothn",
    "Failing to plan is planning to fail",
    "Faint heart never won fair lady",
    "Fair exchange is no robbery",
    "Faith will move mountains",
    "Familiarity breeds contempt",
    "Fat is in the fire",
    "Feed a cold and starve a fever",
    "Female of the species is more deadly than the male",
    "Fight fire with fire",
    "Fight the good fight",
    "Finders keepers, losers weepers",
    "Fine words butter no parsnips",
    "Fire is a good servant but a bad master",
    "First come, first served",
    "First impressions are the most lasting",
    "First things first",
    "Fish always stink from the head down",
    "Fish and guests smell after three days",
    "Flattery will get you nowhere",
    "Fool and his money are soon parted",
    "Fools rush in where angels fear to tread",
    "For everything there is a season",
    "For want of a nail the shoe was lost; for want of a shoe the horse was lost; and for want of a horse the man was lost",
    "Forewarned is forearmed",
    "Forgive and forget",
    "Fortune favours the brave",
    "Friend in need is a friend indeed",
    "From the sublime to the ridiculous is only one step",
    "Genius is an infinite capacity for taking pains",
    "Genius is one percent inspiration, ninety-nine percent perspiration",
    "Give a man a fish and you will feed him for a day...",
    "Give a dog a bad name and hang him",
    "Give a man enough rope and he will hang himself",
    "Give credit where credit is due",
    "Give the Devil his due",
    "Go the extra mile",
    "God helps those who help themselves",
    "Goes without saying - It",
    "Golden key can open any door",
    "Good beginning makes a good ending",
    "Good die young",
    "Good fences make good neighbours",
    "Good man is hard to find",
    "Good talk saves the food",
    "Good things come in small packages",
    "Good things come to those that wait",
    "Grass is always greener on the other side of the fence",
    "Great minds think alike",
    "Great oaks from little acorns grow",
    "Half a loaf is better than no bread",
    "Hand that rocks the cradle rules the world",
    "Handsome is as handsome does",
    "Hard cases make bad law",
    "Hard work never did anyone any harm",
    "Haste makes waste",
    "He that goes a-borrowing, goes a-sorrowing",
    "He who can does, he who cannot, teaches",
    "He who fights and runs away, may live to fight another day",
    "He who hesitates is lost",
    "He who laughs last laughs longest",
    "He who lives by the sword shall die by the sword",
    "He who pays the piper calls the tune",
    "He who sups with the devil should have a long spoon",
    "Hindsight is always twenty-twenty",
    "History repeats itself",
    "Hold with the hare and run with the hounds - You can't",
    "Home is where the heart is",
    "Honesty is the best policy",
    "Honey catches more flies than vinegar",
    "Honour among thievesre's",
    "Hope springs eternal",
    "Horses for courses",
    "House divided against itself cannot stand",
    "House is not a home",
    "The Husband is always the last to know",
    "If anything can go wrong, it will",
    "If a job is worth doing it is worth doing well",
    "If at first you don't succeed try, try and try again",
    "If God had meant us to fly he'd have given us wings",
    "If ifs and ands were pots and pans there'd be no work for tinkers",
    "If it ain't broke, don't fix it",
    "If life deals you lemons, make lemonade",
    "If the cap fits, wear it",
    "If the mountain won't come to Mohammed, then Mohammed must go to the mountain",
    "If the shoe fits, wear it",
    "If wishes were horses, beggars would ride",
    "If you build it they will come",
    "If you can't be good, be careful",
    "If you can't beat em, join em",
    "If you can't stand the heat get out of the kitchen",
    "If you lie down with dogs, you will get up with fleas",
    "If you pay peanuts, you get monkeys",
    "If you want a thing done well, do it yourself",
    "Ignorance is bliss",
    "Imitation is the sincerest form of flattery",
    "In for a penny, in for a pound",
    "In the kingdom of the blind the one eyed man is king",
    "In the midst of life we are in death",
    "Into every life a little rain must fall",
    "It ain't over till the fat lady sings",
    "It goes without saying",
    "It never rains but it pours",
    "It takes a thief to catch a thief",
    "It takes all sorts to make a world",
    "It takes one to know one",
    "It takes two to tango",
    "It's all grist to the mill",
    "It's an ill wind that blows no one any good",
    "It's best to be on the safe side",
    "It's better to give than to receive",
    "It's better to have loved and lost than never to have loved at all",
    "It's better to light a candle than curse the darkness",
    "It's better to travel hopefully than to arrive",
    "It's easy to be wise after the event",
    "It's never too late",
    "It's no use crying over spilt milk",
    "It's no use shutting the stable door after the horse has bolted",
    "It's the early bird that catches the worm",
    "It's the empty can that makes the most noise",
    "It's the singer not the song",
    "It's the squeaky wheel that gets the grease",
    "Jack of all trades, master of none",
    "Jam tomorrow and jam yesterday, but never jam today",
    "Journey of a thousand miles begins with a single step",
    "Judge not, that ye be not judged",
    "Keep your chin up",
    "Keep your friends close and your enemies closer",
    "Keep your powder dry",
    "Know which side your bread is buttered",
    "Labourer is worthy of his hire",
    "Laugh and the world laughs with you, weep and you weep alone",
    "Laughter is the best medicine",
    "Law is an ass",
    "Least said, soonest mended",
    "Leopard cannot change its spots",
    "Less is more",
    "Let bygones be bygones",
    "Let not the sun go down on your wrath",
    "Let sleeping dogs lie",
    "Let the buyer beware",
    "Let the dead bury the dead",
    "Let the punishment fit the crime",
    "Let well alone",
    "Life begins at forty",
    "Life is just a bowl of cherries",
    "Life is what you make it",
    "Life's not all beer and skittles",
    "Lightning never strikes twice in the same place",
    "Like father, like son",
    "Little knowledge is a dangerous thing",
    "Little learning is a dangerous thing",
    "Little of what you fancy does you good",
    "Little pitchers have big ears",
    "Little strokes fell great oaks",
    "Little things please little minds",
    "Live and learn",
    "Live and let live",
    "Live for today for tomorrow never comes",
    "Longest journey starts with a single step",
    "Look before you leap",
    "Love of money is the root of all evil",
    "Love is blind",
    "Love makes the world go round",
    "Love thy neighbour as thyself",
    "Love will find a way",
    "Make hay while the sun shines",
    "Make haste slowly",
    "Make love not war",
    "Man does not live by bread alone",
    "Man is known by his friends",
    "Man who is his own lawyer has a fool for his client",
    "Manners maketh man",
    "Many a good tune played on an old fiddlere's",
    "Many a little makes a mickle",
    "Many a mickle makes a muckle",
    "Many a slip 'twixt cup and lipre's",
    "Many a true word is spoken in jest",
    "Many are called but few are chosen",
    "Many hands make light work",
    "March comes in like a lion, and goes out like a lamb",
    "March winds and April showers bring forth May flowers",
    "Marriages are made in heaven",
    "Marry in haste, repent at leisure",
    "Might is right",
    "Mighty oaks from little acorns grow",
    "Misery loves company",
    "Miss is as good as a mile",
    "Moderation in all things",
    "Monday's child is fair of face",
    "Money doesn't grow on trees",
    "Money is the root of all evil",
    "Money isn't everything",
    "Money makes the world go round",
    "Money talks",
    "More haste, less speed",
    "More the merrier",
    "The more things change, the more they stay the same",
    "There are more ways of killing a cat than choking it with cream",
    "Music has charms to soothe the savage breast",
    "Nature abhors a vacuum",
    "Necessity is the mother of invention",
    "Needs must when the devil drives",
    "Ne'er cast a clout till May be out",
    "Never give a sucker an even break",
    "Never go to bed on an argument",
    "Never judge a book by its cover",
    "Never let the sun go down on your anger",
    "Never look a gift horse in the mouth",
    "Never put off until tomorrow what you can do today",
    "Never rains but it pours",
    "Never speak ill of the dead",
    "Never tell tales out of school",
    "Never too late - It's",
    "New broom sweeps clean",
    "Nine tailors make a man",
    "No accounting for tastesre's",
    "No fool like an old foolre's",
    "No man can serve two masters",
    "No man is an island",
    "No names, no pack-drill",
    "No news is good news",
    "No one can make you feel inferior without your consent",
    "No pain, no gain",
    "No place like homere's",
    "No rest for the wicked",
    "No smoke without firere's",
    "No such thing as a free lunchre's",
    "No such thing as bad publicityre's",
    "No time like the presentre's",
    "No use crying over spilt milk - It's",
    "Nod's as good as a wink to a blind horse",
    "None so blind as those who will not seere's",
    "None so deaf as those who will not hearre's",
    "Nothing new under the sun",
    "Nothing is certain but death and taxes",
    "Nothing succeeds like success",
    "Nothing ventured, nothing gained",
    "Oil and water don't mix",
    "Old soldiers never die, they simply fade away.",
    "Once a thief, always a thief",
    "Once bitten, twice shy",
    "One good turn deserves another",
    "One half of the world does not know how the other half lives",
    "One hand washes the other",
    "One man's meat is another man's poison",
    "One might as well be hanged for a sheep as a lamb",
    "One law for the rich and another law for the poor",
    "One swallow does not make a summer",
    "One volunteer is worth ten pressed men",
    "One year's seeding makes seven years weeding",
    "Only fools and horses work",
    "Opera ain't over till the fat lady sings",
    "Opportunity never knocks twice at any man's door",
    "Opposites attract",
    "Ounce of prevention is worth a pound of cure n",
    "Out of sight, out of mind",
    "Out of the frying pan into the fire",
    "Paddle your own canoe",
    "Parsley seed goes nine times to the Devil",
    "Patience is a virtue",
    "Pearls of wisdom",
    "Pen is mightier than sword",
    "Penny saved is a penny earned",
    "Penny wise and pound foolish",
    "People who live in glass houses shouldn't throw stones",
    "Person is known by the company he keeps",
    "Physician, heal thyself",
    "Picture paints a thousand words",
    "Place for everything and everything in its place",
    "Poor workman always blames his tools",
    "Possession is nine points of the law",
    "Power corrupts; absolute power corrupts absolutely",
    "Practice makes perfect",
    "Practice what you preach",
    "Prevention is better than cure",
    "Price of liberty is eternal vigilance",
    "Pride comes before a fall",
    "Procrastination is the thief of time",
    "Problem shared is a problem halved",
    "Proof of the pudding is in the eating",
    "Prophet is not recognized in his own land",
    "Put the cart before the horse",
    "Put your best foot forward",
    "Put your shoulder to the wheel",
    "Quid pro quo",
    "Rain before seven, fine before eleven",
    "Red sky at night shepherd's delight; red sky in the morning, shepherd's warning",
    "Revenge is a dish best served cold",
    "Revenge is sweet",
    "Rising tide lifts all boats",
    "Road to hell is paved with good intentions",
    "Rob Peter to pay Paul",
    "Rolling stone gathers no moss",
    "Rome wasn't built in a day",
    "See a pin and pick it up, all the day you'll have good luck; see a pin and let it lie, bad luck you'll have all day",
    "See no evil, hear no evil, speak no evil",
    "Seeing is believing",
    "Seek and you shall find",
    "Set a thief to catch a thief",
    "Share and share alike",
    "Shoemaker's son always goes barefoot",
    "Shrouds have no pockets",
    "Silence is golden",
    "Slow but sure",
    "Soft answer turneth away wrath",
    "Softly, softly, catchee monkey",
    "Spare the rod and spoil the child",
    "Speak as you find",
    "Speak softly and carry a big stick",
    "Squeaky wheel gets the grease",
    "Sticks and stones may break my bones, but words will never hurt me",
    "Still waters run deep",
    "Stitch in time saves nine",
    "Strike while the iron is hot",
    "Stupid is as stupid does",
    "Success has many fathers, while failure is an orphan",
    "Take care of the pennies and the pounds will take care of themselves",
    "It Takes a thief to catch a thief",
    "It Takes all sorts to make a world",
    "It Takes one to know one",
    "Talk is cheap",
    "Talk of the Devil, and he is bound to appear",
    "Tell the truth and shame the Devil",
    "That which does not kill us makes us stronger",
    "The age of miracles is past",
    "The apple never falls far from the tree",
    "The best defense is a good offence",
    "The best is the enemy of the good",
    "The best-laid schemes of mice and men gang aft agley",
    "The best things in life are free",
    "The bigger, the better",
    "The bigger they are, the harder they fall",
    "The bottom line is the bottom line",
    "The boy is father to the man",
    "The bread always falls buttered side down",
    "The child is father to the man",
    "The cobbler always wears the worst shoes",
    "The course of true love never did run smooth",
    "The customer is always right",
    "The darkest hour is just before the dawn",
    "The Devil has all the best tunes",
    "The Devil is in the details",
    "The Devil looks after his own",
    "The Devil makes work for idle hands to do",
    "The Devil take the hindmost",
    "The early bird catches the worm",
    "The empty can that makes the most noise - It's",
    "The end justifies the means",
    "The English are a nation of shopkeepers",
    "The exception which proves the rule",
    "The fat is in the fire",
    "The female of the species is more deadly than the male",
    "The good die young",
    "The grass is always greener on the other side of the fence",
    "The hand that rocks the cradle rules the world",
    "The husband is always the last to know",
    "The labourer is worthy of his hire",
    "The law is an ass",
    "The leopard does not change his spots",
    "The longest journey starts with a single step",
    "The more the merrier",
    "The more things change, the more they stay the same",
    "The opera ain't over till the fat lady sings",
    "The pen is mightier than sword",
    "The price of liberty is eternal vigilance",
    "The proof of the pudding is in the eating",
    "The rich get richer and the poor get poorer",
    "The road to hell is paved with good intentions",
    "The shoemaker's son always goes barefoot",
    "It's the singer not the song",
    "It's the squeaky wheel that gets the grease",
    "The truth will out",
    "The wages of sin is death",
    "The way to a man's heart is through his stomach",
    "The whole is greater than the sum of the parts",
    "There are more ways of killing a cat than choking it with cream",
    "There are none so blind as those, that will not see",
    "There are two sides to every question",
    "There but for the grace of God, go I",
    "There's a sucker born every minute",
    "There's a time and a place for everything",
    "There's an exception to every rule",
    "There's always more fish in the sea",
    "There's honour among thieves",
    "There's many a good tune played on an old fiddle",
    "There's many a slip 'twixt cup and lip",
    "There's more than one way to skin a cat",
    "There's no accounting for tastes",
    "There's no fool like an old fool",
    "There's no place like home",
    "There's no smoke without fire",
    "There's no such thing as a free lunch",
    "There's no such thing as bad publicity",
    "There's no time like the present",
    "There's none so blind as those who will not see",
    "There's none so deaf as those who will not hear",
    "There's nowt so queer as folk",
    "There's one born every minute",
    "There's one law for the rich and another law for the poor",
    "There's safety in numbers",
    "They that sow the wind, shall reap the whirlwind",
    "Thing of beauty is a joy forever",
    "Third time lucky",
    "Those who do not learn from history are doomed to repeat it",
    "Those who live in glass houses shouldn't throw stones",
    "Those who sleep with dogs will rise with fleas",
    "Trouble shared is a trouble halved",
    "Thou shalt not kill",
    "Time and tide wait for no man",
    "Time flies",
    "Time is a great healer",
    "Time is money",
    "Time will tell",
    "'tis better to have loved and lost, than never to have loved at all",
    "To err is human; to forgive divine",
    "To every thing there is a season",
    "To the victor go the spoils",
    "To travel hopefully is a better thing than to arrive",
    "Tomorrow is another day",
    "Tomorrow never comes",
    "Too many cooks spoil the broth",
    "Truth is stranger than fiction",
    "Truth will out",
    "Two blacks don't make a white",
    "Two heads are better than one",
    "Two is company, but three's a crowd",
    "Two sides to every question - There are",
    "Two wrongs don't make a right",
    "Variety is the spice of life",
    "Virtue is its own reward",
    "Volunteer is worth twenty pressed men",
    "Wages of sin is death - The",
    "Walls have ears",
    "Walnuts and pears you plant for your heirs",
    "Waste not want not",
    "Watched pot never boils",
    "Way to a man's heart is through his stomach - The",
    "What can't be cured must be endured",
    "What goes up must come down",
    "What you lose on the swings you gain on the roundabouts",
    "What's sauce for the goose is sauce for the gander",
    "When in Rome, do as the Romans do",
    "When the cat's away the mice will play",
    "When the going gets tough, the tough get going",
    "When the oak is before the ash, then you will only get a splash; when the ash is before the oak, then you may expect a soak",
    "What the eye doesn't see, the heart doesn't grieve over",
    "Where there's a will there's a way",
    "Where there's muck there's brass",
    "Wherever you wander, there's no place like home",
    "While there's life there's hope",
    "Whole is greater than the sum of the parts - The",
    "Whom the Gods love die young",
    "Why keep a dog and bark yourself?",
    "Woman is only a woman, but a good cigar is a smoke",
    "Woman's place is in the home",
    "Woman's work is never done",
    "Women and children first",
    "Wonders will never cease",
    "Word to the wise is enough",
    "Work expands so as to fill the time available",
    "Worrying never did anyone any good",
    "You are never too old to learn",
    "You are what you eat",
    "You can catch more flies with honey than with vinegar",
    "You can choose your friends but you can't choose your family",
    "You can have too much of a good thing",
    "You can lead a horse to water, but you can't make it drink",
    "You can't have your cake and eat it too",
    "You can't get blood out of a stone",
    "You can't get blood out of a turnip",
    "You can't hold with the hare and run with the hounds",
    "You can't make a silk purse out of a sow's ear",
    "You can't make an omelette without breaking eggs",
    "You can't make bricks without straw",
    "You can't run with the hare and hunt with the hounds",
    "You can't take it with you [when you die]",
    "You can't teach an old dog new tricks",
    "You can't judge a book by its cover",
    "You can't win them all",
    "You catch more flies with honey than with vinegar",
    "You pays your money and you takes your choice",
    "You reap what you sow",
    "You win some, you lose some",
    "Youth is wasted on the young"












]

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]


eng_Latn - hin_Deva
eng_Latn: A bad penny always turns up
hin_Deva: एक बुरा पैसा हमेशा सामने आता है।
eng_Latn: A barking dog never bites
hin_Deva: भौंकने वाला कुत्ता कभी नहीं काटता।
eng_Latn: A bird in the hand is worth two in the bush
hin_Deva: हाथ में एक पक्षी झाड़ी में दो के बराबर होता है।
eng_Latn: A cat may look at a king
hin_Deva: एक बिल्ली एक राजा को देख सकती है
eng_Latn: A chain is only as strong as its weakest link
hin_Deva: एक जंजीर अपनी सबसे कमजोर कड़ी जितनी ही मजबूत होती है।
eng_Latn: A change is as good as a rest
hin_Deva: एक बदलाव उतना ही अच्छा है जितना कि आराम करना।
eng_Latn: A dog is a man's best friend
hin_Deva: कुत्ता इंसान का सबसे अच्छा दोस्त होता है।
eng_Latn: A drowning man will clutch at a straw
hin_Deva: एक डूबता हुआ आदमी एक पुआल को पकड़ेगा
eng_Latn: A fish always rots from the head down
hin_Deva: एक मछली हमेशा सिर से नीचे तक सड़ती है।
eng_Latn: A fool and his money are soon parted
hin_Deva: एक मूर्ख और उसका पैसा जल्द ही अलग हो जाते हैं।
eng_Latn: A friend in ne

### Indic to English Example

In [ ]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")

ip = IndicProcessor(inference=True)

hi_sents = [
    'जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल',
    'मेरे भाई की माँ की बेटी की इकलौती भाई की पत्नी के बेटे की माँ मेरी भाभी है',
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "उसके पास बहुत सारी सम्पति हैं, जिसे उसने अपने दादा-परदादा से विरासत में पाया।",
    "उसके पास बहुत सारी ज़मीन है, जिसे उसने अपने दादा-परदादा से विरासत में पाया।",
    "उसके पास बहुत सारी पुरानी कलम हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
    "आप पर मेरा ₹1234 बकाया है।  पेटीएम करें।",
    "वह समुद्र तट पर समुद्री गोले बेचती है।",
    "मेरे चचेरे भाइयों ने मेरे भाई की शादी में नृत्य किया",
    "मेरे भाभी को चॉकलेट पसंद है",
    "मेरे भाई को चॉकलेट खाना बहुत पसंद है।",
    "मेरे मामा की जूतों की दुकान है।",
    "मेरी नानी हमारे लिए स्वादिष्ट खाना बनाती हैं।",
    "मेरी ताई जी रोज जिम जाती हैं।",
    "मेरे जीजा ने मेहमानों के लिए जगह साफ करने में मेरी मदद की।",
    "मेरी बड़ी माँ ने मेहमानों के लिए जगह साफ करने में मेरी मदद की",
    "मेरी दादी हमारे लिए स्वादिष्ट भोजन बनाती हैं।",
    "मेरे दादा एक बुद्धिमान व्यक्ति हैं।",
    "हम कल अपनी दादी के घर गए थे।",
    "हम आज अपनी चाची के पास जाएँगे।",
    "मेरी नीस इतनी प्यारी है कि जब भी मैं उसे देखता हूं तो मैं उसका चेहरा निचोड़ना चाहता हूं।",
    "मेरे भतीजे ने पिछले हफ्ते स्कूल जाना शुरू कर दिया।"
    "यह बच्चा मेरी आँखों का तारा है",
    "उसकी क्या बात कर रहे हो, वह तो मेरे खून का प्यासा हो गया है।",
    "आतंकवादियों को देखकर मेरा तो खून ठण्डा पड़ गया।",
    "आई.पी.एस पास करके दीक्षा ने सचमुच गढ़ फतह कर लिया।",
    "कार्तिक गधे को बाप बनाना अच्छी तरह जानता हैं।",
    "अभी आपको मेरी बात समझ में नहीं आ रही, जब जूते पड़ेंगे तब समझ में आएगी।",
    "जहाँ चाह, वहाँ राह।",
    "हर कड़ी में राह होती है।",
    "काम का बोझ भारी होता है।",
    "रोम एक दिन में नहीं बना था।",
    "अब पछताए होत क्या जब चिड़ीया चुग गई खेत।",
    "हर चमकते चीज सोना नहीं होती।",
    "दो गलत सही नहीं बनाते।",
    "जहाँ धूम है, वहाँ आग होगी।",
    "विद्यार्थी स्कूल जाने का समय है। यह बहुत महत्वपूर्ण है कि आप समय पर स्कूल पहुंचें ताकि आप अपने पढ़ाई में निरंतरता बनाए रख सकें। स्कूल जाना आपके शिक्षा के लिए आवश्यक है और यह आपके भविष्य को सुनिश्चित करने में मदद कर सकता है। इसलिए, आप आज के दिन को सकारात्मकता के साथ शुरू करें और स्कूल के लिए समय पर निकलें। शुभ कामनाएं!",
    "शुभ प्रभात! आपका दिन सुखमय और सफल हो",
    "दिव्यांग व्यक्तियों की मदद करना सामाजिक जिम्मेदारी है।",
    "इन व्यक्तियों को समाज में समाहित करने और उन्हें समर्थ बनाने के लिए हमें सकारात्मक कदम उठाने चाहिए",
    "दिव्यांग व्यक्तियों के साथ सहानुभूति और समर्थन बनाए रखना अत्यंत महत्वपूर्ण है।",
    "उन्हें समाज में पूरी तरह से शामिल करना और उनकी सामाजिक समाहिति को बढ़ावा देना।",
    "उन्हें उच्च शिक्षा और प्रशिक्षण के अवसरों का भी लाभ दिलाना।",
    "सरकार और सामाजिक संगठनों से उनके लिए सुविधाएं और योजनाएं मांगना और उन्हें इनका लाभ उठाने में मदद करना।",
    "उन्हें रोजगार के अवसरों को सुझाना और उन्हें समर्थ बनाने के लिए तकनीकी और व्यावासायिक प्रशिक्षण प्रदान करना।",
    "सामाजिक संगठनों में सक्रियता दिखाना और दिव्यांगता के मुद्दों पर जागरूकता फैलाना।",
    "यह सभी कदम सामूहिक रूप से समृद्धि और समाज में समाहिति की दिशा में सहायक हो सकते हैं।",
    "कनक कनक ते सौ गुनी, मादकता अधिकाय। वा खाए बौराय जग, या पाए बौराय।",
    "उसकी मुस्कान सूर्य की किरणों की तरह चमक रही थी।",
    "समय एक अजीब सा दरवाज़ा है, जो कभी फिर नहीं खुलता।",
    "बादल बढ़ रहे हैं, बारिश बरस रही है।",
    "बस बुढ़िया बैठी बाल कांपा कर बोली।",
    "बाग़ के फूलों ने रात को महका दिया।",
    "गंगा जी की धारा, स्वर्ग से भी प्यारी है।",
    "सुरमा स्वर्ग से सजीवनी लेकर आई।",
    "उसकी आँखों में आसमान का रंग और उसके बोल में दरिया की गहराई थी",
    "हनुमान की पूंछ में लगन ना पाई आग, लंका सारी जल गई, गए निशाचर भग",


    #################Muhavre################
    "अपने काम के लिए गधे को भी बाप बनाना पड़ता है",
    "कथनी और करनी में बहुत फर्क होता है",
    "उसने अपनी मेहनत और लगन से एक सिक्का भी सोना बना दिया।",
    # "उसके पास इतने धैर्य और संवेदनशीलता है कि वह हर रास्ते की धूल सोना बना देता है।",
    "स्थिरता और संयम के बिना, कोई भी उच्चाधिकार को हाथ में नहीं ले सकता।",
    # "उसने अपने विद्युत वाहन को सुरंग के समान बना दिया और उसका संचालन किया।",
    # "सफलता के लिए, वह हमेशा स्थिरता और धैर्य की आवश्यकता को समझता है।",
    # "उसने अपने सपनों की ऊंचाई पर पहुंचने के लिए ज़िद्द की हैरान कर दी।",
    # "जीवन के हर आधार पर वह अपने लक्ष्यों को प्राप्त करने के लिए मनोबल का उपयोग करता है।",
    "वह ने अपनी कलम से विचारों की उड़ान भर दी, जिससे उसके सपने हकीकत में बदल गए।",
    # "उसने अपने कार्य में स्थिरता का अद्वितीय उपयोग किया, जिससे वह हर कठिनाई को पार कर सका।",
    # "उसने अपने नेतृत्व का अद्वितीय साक्षात्कार किया, जिससे वह अपने विश्वास को अद्वितीय रूप से प्रदर्शित कर सका।",
    # "उसकी मेहनत ने उसे अच्छे दिनों की रोशनी दिखाई।",
    # "उसका दिल बड़ा है, लेकिन उसकी ताकत उसके दिमाग में है।",
    "उसका मुँह काला हो गया, जब उसका जूता उसके सर पर पड़ा।",
    "वह ताल ठोक रहा है, जबकि खुद की पूरी लाइन उसके सामने है।",
    "उसकी ज़बान में तलवार है, लेकिन उसके हाथ में उसकी ताकत नहीं है।",
    # "उसने बिल्लियों को दूध पिलाया, लेकिन बिल्लियों के मुँह में मारने का हाल नहीं।",
    # "उसका मुँह तोड़ देने के बाद, उसकी आँखें खुद नहीं रुकी।",
    # "उसने उसे खाली हाथ भेजा, लेकिन उसने खुद को पूरी जीत का मुँह दिखाया।",
    "वह उसके पीछे पड़ा है, जैसे कोई कुत्ता भागता है।",
    # "उसने अपने पैरों में खुद को कुंवारा बनाया, जबकि उसके सर पर उसकी शादी का झोंका लटका है।",


    ]
# hi_sents = hi_translations
src_lang, tgt_lang = "hin_Deva", "eng_Latn"
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, en_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_en_tokenizer, indic_en_model

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]


hin_Deva - eng_Latn
hin_Deva: जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल
eng_Latn: When life is out of control, make lips round, make lips round, make whistles round, say all is well.
hin_Deva: मेरे भाई की माँ की बेटी की इकलौती भाई की पत्नी के बेटे की माँ मेरी भाभी है
eng_Latn: My brother's mother's daughter's only brother's wife's son's mother is my sister-in-law
hin_Deva: जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।
eng_Latn: When I was young, I used to go to the park every day.
hin_Deva: उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।
eng_Latn: She has a lot of old books, which she inherited from her grandparents.
hin_Deva: उसके पास बहुत सारी सम्पति हैं, जिसे उसने अपने दादा-परदादा से विरासत में पाया।
eng_Latn: He has a lot of wealth, which he inherited from his grandparents.
hin_Deva: उसके पास बहुत सारी ज़मीन है, जिसे उसने अपने दादा-परदादा से विरासत में पाया।
eng_Latn: He owns a lot of land, which 

### Indic to Indic Example


In [ ]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)

hi_sents = [
    'जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल',
    'मेरे भाई की माँ की बेटी की इकलौती भाई की पत्नी के बेटे की माँ मेरी भाभी है',
    "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
    "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
    "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
    "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
    "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
    "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
    "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
    "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
    "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
    "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
    "आप पर मेरा ₹1234 बकाया है।  पेटीएम करें।",
    "वह समुद्र तट पर समुद्री गोले बेचती है।",
    "मेरे चचेरे भाइयों ने मेरे भाई की शादी में नृत्य किया",
    "मेरे भाभी को चॉकलेट पसंद है",
    "मेरे भाई को चॉकलेट खाना बहुत पसंद है।",
    "मेरे मामा की जूतों की दुकान है।",
    "मेरी नानी हमारे लिए स्वादिष्ट खाना बनाती हैं।",
    "मेरी ताई जी रोज जिम जाती हैं।",
    "मेरे जीजा ने मेहमानों के लिए जगह साफ करने में मेरी मदद की।",
    "मेरी बड़ी माँ ने मेहमानों के लिए जगह साफ करने में मेरी मदद की",
    "मेरी दादी हमारे लिए स्वादिष्ट भोजन बनाती हैं।",
    "मेरे दादा एक बुद्धिमान व्यक्ति हैं।",
    "हम कल अपनी दादी के घर गए थे।",
    "हम आज अपनी चाची के पास जाएँगे।",
    "मेरी नीस इतनी प्यारी है कि जब भी मैं उसे देखता हूं तो मैं उसका चेहरा निचोड़ना चाहता हूं।",
    "मेरे भतीजे ने पिछले हफ्ते स्कूल जाना शुरू कर दिया।"
    "यह बच्चा मेरी आँखों का तारा है",
    "उसकी क्या बात कर रहे हो, वह तो मेरे खून का प्यासा हो गया है।",
    "आतंकवादियों को देखकर मेरा तो खून ठण्डा पड़ गया।",
    "आई.पी.एस पास करके दीक्षा ने सचमुच गढ़ फतह कर लिया।",
    "कार्तिक गधे को बाप बनाना अच्छी तरह जानता हैं।",
    "अभी आपको मेरी बात समझ में नहीं आ रही, जब जूते पड़ेंगे तब समझ में आएगी।",
    "जहाँ चाह, वहाँ राह।",
    "हर कड़ी में राह होती है।",
    "काम का बोझ भारी होता है।",
    "रोम एक दिन में नहीं बना था।",
    "अब पछताए होत क्या जब चिड़ीया चुग गई खेत।",
    "हर चमकते चीज सोना नहीं होती।",
    "दो गलत सही नहीं बनाते।",
    "जहाँ धूम है, वहाँ आग होगी।",
    "विद्यार्थी स्कूल जाने का समय है। यह बहुत महत्वपूर्ण है कि आप समय पर स्कूल पहुंचें ताकि आप अपने पढ़ाई में निरंतरता बनाए रख सकें। स्कूल जाना आपके शिक्षा के लिए आवश्यक है और यह आपके भविष्य को सुनिश्चित करने में मदद कर सकता है। इसलिए, आप आज के दिन को सकारात्मकता के साथ शुरू करें और स्कूल के लिए समय पर निकलें। शुभ कामनाएं!",
    "शुभ प्रभात! आपका दिन सुखमय और सफल हो",
    "दिव्यांग व्यक्तियों की मदद करना सामाजिक जिम्मेदारी है।",
    "इन व्यक्तियों को समाज में समाहित करने और उन्हें समर्थ बनाने के लिए हमें सकारात्मक कदम उठाने चाहिए",
    "दिव्यांग व्यक्तियों के साथ सहानुभूति और समर्थन बनाए रखना अत्यंत महत्वपूर्ण है।",
    "उन्हें समाज में पूरी तरह से शामिल करना और उनकी सामाजिक समाहिति को बढ़ावा देना।",
    "उन्हें उच्च शिक्षा और प्रशिक्षण के अवसरों का भी लाभ दिलाना।",
    "सरकार और सामाजिक संगठनों से उनके लिए सुविधाएं और योजनाएं मांगना और उन्हें इनका लाभ उठाने में मदद करना।",
    "उन्हें रोजगार के अवसरों को सुझाना और उन्हें समर्थ बनाने के लिए तकनीकी और व्यावासायिक प्रशिक्षण प्रदान करना।",
    "सामाजिक संगठनों में सक्रियता दिखाना और दिव्यांगता के मुद्दों पर जागरूकता फैलाना।",
    "यह सभी कदम सामूहिक रूप से समृद्धि और समाज में समाहिति की दिशा में सहायक हो सकते हैं।",
    "कनक कनक ते सौ गुनी, मादकता अधिकाय। वा खाए बौराय जग, या पाए बौराय।",
    "उसकी मुस्कान सूर्य की किरणों की तरह चमक रही थी।",
    "समय एक अजीब सा दरवाज़ा है, जो कभी फिर नहीं खुलता।",
    "बादल बढ़ रहे हैं, बारिश बरस रही है।",
    "बस बुढ़िया बैठी बाल कांपा कर बोली।",
    "बाग़ के फूलों ने रात को महका दिया।",
    "गंगा जी की धारा, स्वर्ग से भी प्यारी है।",
    "सुरमा स्वर्ग से सजीवनी लेकर आई।",
    "उसकी आँखों में आसमान का रंग और उसके बोल में दरिया की गहराई थी",
    "हनुमान की पूंछ में लगन ना पाई आग, लंका सारी जल गई, गए निशाचर भग"

]
src_lang, tgt_lang = "hin_Deva", "ory_Orya"
mr_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)

print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(hi_sents, mr_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
del indic_indic_tokenizer, indic_indic_model

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]


hin_Deva - ory_Orya
hin_Deva: जब लाइफ़ हो आउट ऑफ कंट्रोल होठों को करके गोल होठों को करके गोल सीटी बजाके बोल आल इज़ वेल
ory_Orya: ଯେତେବେଳେ ଜୀବନ ନିଯ଼ନ୍ତ୍ରଣ ବାହାରେ ଥାଏ, ଓଠକୁ ଗୋଲ କରିଦିଅନ୍ତୁ, ଓଠକୁ ଗୋଲ କରିଦିଅନ୍ତୁ, ହୁଇସିଲ୍ ବାଜନ୍ତୁ, ସବୁକିଛି ଠିକ୍ ଅଛି।
hin_Deva: मेरे भाई की माँ की बेटी की इकलौती भाई की पत्नी के बेटे की माँ मेरी भाभी है
ory_Orya: ମୋ ଭାଇଙ୍କ ମା 'ଙ୍କ ଝିଅର ଏକମାତ୍ର ଭାଇଙ୍କ ପତ୍ନୀଙ୍କ ପୁଅର ମାଆ ମୋ ଭିଣୋଇ।
hin_Deva: जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।
ory_Orya: ଯେତେବେଳେ ମୁଁ ଛୋଟ ଥିଲି, ମୁଁ ପ୍ରତିଦିନ ପାର୍କକୁ ଯାଉଥିଲି।
hin_Deva: उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।
ory_Orya: ତାଙ୍କ ପାଖରେ ଅନେକ ପୁରୁଣା ପୁସ୍ତକ ଅଛି, ଯାହା ସେ ତାଙ୍କ ଜେଜେବାପା-ଜେଜେମାଙ୍କଠାରୁ ଉତ୍ତରାଧିକାରରେ ପାଇଥିଲେ।
hin_Deva: मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।
ory_Orya: ମୁଁ ବୁଝିପାରୁନାହିଁ ଯେ ମୁଁ ମୋ ସମସ୍ଯ଼ାର ସମାଧାନ କିପରି ପାଇବି।
hin_Deva: वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।
ory_Orya: ସେ ବହୁତ ପରିଶ୍ରମୀ ଏବଂ ବୁଦ୍ଧିମାନ, ତେଣୁ ତାଙ୍କୁ ସମସ୍

In [ ]:
# from google.colab import drive

In [ ]:
# %cd content/drive/MyDrive/IndicTrans2/

In [ ]:
# import os

# # Get the current working directory
# current_directory = os.getcwd()

# # Print the current working directory
# print("Current working directory:", current_directory)

In [ ]:
# import os

# # Get the current working directory
# current_directory = os.getcwd()

# # List all the contents of the current directory
# directory_contents = os.listdir(current_directory)

# # Print the contents of the directory
# print("Contents of the current directory:")
# for item in directory_contents:
#     print(item)

In [ ]:
# import os

# # Specify the name of the folder you want to navigate to
# folder_name = "IndicTrans2"

# # Get the current working directory
# current_directory = os.getcwd()

# # Construct the path to the folder you want to navigate to
# folder_path = os.path.join(current_directory, folder_name)

# # Change the current working directory to the specified folder
# os.chdir(folder_path)

# # Confirm that you are now in the specified folder
# print("Current working directory:", os.getcwd())


In [ ]:
# import os

# # Get the current working directory
# current_directory = os.getcwd()

# # List all the contents of the current directory
# directory_contents = os.listdir(current_directory)

# # Print the contents of the directory
# print("Contents of the current directory:")
# for item in directory_contents:
#     print(item)

In [ ]:
# import os

# # Navigate back to the parent directory
# os.chdir('..')

# # Confirm that you are now in the parent directory
# print("Current working directory:", os.getcwd())


In [ ]:
# !git clone https://github.com/dikshaseth/NLP-Class-Project.git

In [ ]:
# %cd NLP-Class-Project

In [ ]:
# !git add .
# !git commit -m "1st commit"

In [ ]:
# !git config --global user.email "dikshaseth@iisc.ac.in"
# !git config --global user.name "dikshaseth"

In [ ]:
# !git push origin master